# RAG w/ Langgraph
`12_langgraph_rag.ipynb`

- https://python.langchain.com/docs/tutorials/rag/

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [4]:
from pprint import pprint

# 1. Loader (웹문서)
from langchain_community.document_loaders import WebBaseLoader
from bs4 import SoupStrainer          # 권장
# 또는
from bs4.element import SoupStrainer


loader = WebBaseLoader(
    # 문서 출처 URL
    web_paths=('https://lilianweng.github.io/posts/2023-06-23-agent/', ),
    # 웹페이지 안에서 필요한 정보만 선택
    bs_kwargs={
        'parse_only': SoupStrainer(class_=['post-content']) 
    }
    # header_template={}
)
docs = loader.load()

# 2. Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_docs = splitter.split_documents(docs)
print(len(splitted_docs))

# 3. Embedding Model
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')  # small <-> large

# 4. Vectorstore (지금은 FAISS -> 클라우드-Pinecone)
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(splitted_docs, embedding=embedding)

USER_AGENT environment variable not set, consider setting it to identify your requests.


63


In [5]:
from langchain import hub

prompt = hub.pull('rlm/rag-prompt')

for m in prompt.messages:
    m.pretty_print()

================================ Human Message =================================

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [6]:
# LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4.1', temperature=0)

# State
from langchain_core.documents import Document
from typing_extensions import TypedDict, List

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Node
# 검색 노드
def retrieve(state: State):
    # [ Document * 4 ]
    retrieved_docs = vectorstore.similarity_search(state['question'], k=4)
    
    # Document 객체의 필요없는 정보는 다 빼고, 내용에 해당하는 page_content 만 모아서 넘기면 토큰 절약 가능.
    # context_str = ''
    # for doc in retrieved_docs:
    #     context_str += doc.page_content + '\n------------------------\n'

    # 나머지 return 하지 않은 state 항목들은, 알아서 그대로 감 (question, answer 는 알아서 그대로 나감)
    return { 'context': context_str, }

# 답변 생성노드
def generate(state: State):
    context_str = ''
    for doc in state['context']:
        context_str += doc.page_content + '\n-------------------\n'
    question_with_context = prompt.invoke({'question': state['question'], 'context': state['context']})
    response = llm.invoke(question_with_context)
    return {'answer': response.content}

# Graph
from langgraph.graph import StateGraph, START, END
builder = StateGraph(State)

builder.add_node('retrieve', retrieve)
builder.add_node('generate', generate)

builder.add_edge(START, 'retrieve')
builder.add_edge('retrieve', 'generate')
builder.add_edge('generate', END)

graph = builder.compile()

# 출력
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

In [8]:
# 상태 타입 예시
from typing import TypedDict

class State(TypedDict):
    question: str
    context: str   # retrieve에서 채워줄 키
    answer: str    # generate에서 채울 키

# retrieve 노드
def retrieve(state: State):
    question = state["question"]
    docs = retriever.invoke(question)  # 또는 retriever.get_relevant_documents(question)

    # 컨텐츠만 이어붙여 문자열 컨텍스트 생성
    context_str = "\n\n".join(d.page_content for d in docs) if docs else ""

    # 여기서 반드시 context_str를 반환 키와 맞춰 사용
    return {"context": context_str}

# generate 노드(참고)
def generate(state: State):
    prompt = f"질문:\n{state['question']}\n\n관련 문맥:\n{state['context']}\n\n정확하고 간결히 답하세요."
    resp = llm.invoke(prompt)  # 사용 중인 LLM 호출로 대체
    return {"answer": resp.content if hasattr(resp, "content") else str(resp)}


In [9]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)
builder.add_node("retrieve", retrieve)
builder.add_node("generate", generate)

builder.set_entry_point("retrieve")
builder.add_edge("retrieve", "generate")
builder.add_edge("generate", END)

graph = builder.compile()


*메세지 스트리밍*

In [22]:
# LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4.1', temperature=0)

# State
from langchain_core.documents import Document
from typing_extensions import TypedDict, List

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Node
# 검색 노드
def retrieve(state: State):
    # [ Document * 4 ]
    retrieved_docs = vectorstore.similarity_search(state['question'], k=4)
    
    # Document 객체의 필요없는 정보는 다 빼고, 내용에 해당하는 page_content 만 모아서 넘기면 토큰 절약 가능.
    context_str = ''
    for doc in retrieved_docs:
        context_str += doc.page_content + '\n------------------------\n'

    # 나머지 return 하지 않은 state 항목들은, 알아서 그대로 감 (question, answer 는 알아서 그대로 나감)
    return { 'context': context_str, }

# 답변 생성노드
def generate(state: State):
    question_with_context = prompt.invoke({'question': state['question'], 'context': state['context']})
    response = llm.invoke(question_with_context)
    return {'answer': response.content}

# Graph
from langgraph.graph import StateGraph, START, END
builder = StateGraph(State)

builder.add_node('retrieve', retrieve)
builder.add_node('generate', generate)

builder.add_edge(START, 'retrieve')
builder.add_edge('retrieve', 'generate')
builder.add_edge('generate', END)

graph = builder.compile()

# 출력
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

## RAG + 
- Metadata 편집
-Query 분석 - 보완
-사용자 질문을 재분배해야 한다. 다른 문서가 나올 수 있다. 


In [20]:
# 문서 63개중 1/3 지점
third = len(splitted_docs) // 3

# metadata에 'section' 추가중
for idx, doc in enumerate(splitted_docs):
    if idx < third:
        doc.metadata['section'] = '전반부'
    elif idx < third *2:
        doc.metadata['section'] = '중반부'
    else:
        doc.metadata['section'] = '후반부'

splitted_docs[0].metadata
vectorstore = FAISS.from_documents(splitted_docs, embedding=embedding)

In [23]:
# State를 더 빡빡하게 정의하기 위해
# StructuredOutPut
from typing import Literal
from typing_extensions import Annotated 

class Search(TypedDict):
    """Vectorstore Search Query"""
    # 1. 타입, 2. NOT NULL, 3. 설명
    query: Annotated[str, ...,'Search query to run']
    section: Annotated [
        Literal['전반부','중반부','후반부'],
        ...,
        'Section to query'
    ]

class MyState(TypedDict):
    question: str
    query: Search
    context:List[Document]
    answer: str

In [ ]:
# Node
def analyze_query(state: MyState):
    #Search 클래스에 맞춰 사용자 question을 {query, section}으로 바꿈
    s_llm = llm.with_structured_output(Search)
    query = s_llm.invoke(state['question'])
    print(query)    
    return {'query':query}
def retrieve(state: MyState):
    query = state['query']
    docs = vectorstore.similarity_search(
        query['query'],
        # llm이 판단한 section과 실제 문서조각의 section이 맞을 경우에만 검색.
        filter = lambda doc: doc.metadata.get('section') == query['section']
    )
    return {'context':docs}
def generate(state: MyState):
    # Token 아끼기 위해, 내용만 추려서 문자열로 만들기
    doc_str = ''
    for doc in state['context']:
        doc_str += doc.page_content + '\n======================\n'
    question_with_context = prompt.invoke({'question':state['question'],'context':doc_str})
    res = llm.invoke(question_with_context)
    return {'answer':res.content}
builder = StateGraph(MyState)
builder.add_node('analye_query',analyze_query)
builder.add_node('retrieve',retrieve)
builder.add_node('generate',generate)

builder.add_edge(START, 'analyze_query')
builder.add_edge('analyze_query','retrieve')
builder.add_edge('retrieve','generate')
builder.add_edge('generate',)


